# Regression
In this example we are building a model that predicts house prices in Boston  
Dataset: 


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [1]:
from hana_automl.automl import AutoML
import pandas as pd
from hana_ml.dataframe import ConnectionContext
from hana_automl.storage import Storage

Let's get used to the dataset

In [2]:
test_df = pd.read_csv('./docs/source/datasets/boston_test_data.csv')
df = pd.read_csv('./docs/source/datasets/boston_data.csv')
test_df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,0.07886,80.0,4.95,0.0,0.411,7.148,27.7,5.1167,4.0,245.0,19.2,396.90,3.56
1,0.08873,21.0,5.64,0.0,0.439,5.963,45.7,6.8147,4.0,243.0,16.8,395.56,13.45
2,1.38799,0.0,8.14,0.0,0.538,5.950,82.0,3.9900,4.0,307.0,21.0,232.60,27.71
3,0.30347,0.0,7.38,0.0,0.493,6.312,28.9,5.4159,5.0,287.0,19.6,396.90,6.15
4,0.22927,0.0,6.91,0.0,0.448,6.030,85.5,5.6894,3.0,233.0,17.9,392.74,18.80


Pass credentials to the database.

In [3]:
cc = ConnectionContext(address='localhost', port=39015, user='DEVELOPER')

In [4]:
automl = AutoML(connection_context=cc)

In [5]:
automl.fit(
    df=df,
    task=None, # library will try to determine task
    steps=10,
    target='medv',
    table_name='REGRESSION', # optional
    id_column='ID', # pass None if no ID column in dataset
    verbosity=1
)

Recreating table REGRESSION with data from dataframe
100%|██████████| 1/1 [00:00<00:00,  1.84it/s]
Task: reg
All iterations completed successfully!
Starting model accuracy evaluation on the validation data!


Save model

In [6]:
storage = Storage(connection_context=cc, schema='DEVELOPER')
automl.model.name = "boston" # don't forget to specify the name
storage.save_model(automl=automl)
storage.list_models()
storage.list_preprocessors()

,MODEL,VERSION,JSON
0,boston,1,"{""num_strategy"": [""mean"", ""median"", ""zero""], ""..."
1,boston,2,"{""num_strategy"": [""mean"", ""median"", ""zero""], ""..."
2,boston,3,"{""num_strategy"": [""mean"", ""median"", ""zero""], ""..."
3,boston,4,"{""num_strategy"": [""mean"", ""median"", ""zero""], ""..."
4,boston,5,"{""num_strategy"": [""mean"", ""median"", ""zero""], ""..."


In [7]:
storage.list_models()

,NAME,VERSION,LIBRARY,CLASS,JSON,TIMESTAMP,MODEL_STORAGE_VER
0,boston,1,PAL,hana_ml.algorithms.pal.trees.GradientBoostingR...,"{""model_attributes"": {""n_estimators"": 280, ""su...",2021-05-20 14:40:32,1
1,boston,2,PAL,hana_ml.algorithms.pal.trees.GradientBoostingR...,"{""model_attributes"": {""n_estimators"": 280, ""su...",2021-05-20 14:45:29,1
2,boston,3,PAL,hana_ml.algorithms.pal.trees.GradientBoostingR...,"{""model_attributes"": {""n_estimators"": 280, ""su...",2021-05-20 14:45:42,1
3,boston,4,PAL,hana_ml.algorithms.pal.trees.GradientBoostingR...,"{""model_attributes"": {""n_estimators"": 294, ""su...",2021-05-20 14:52:14,1
4,boston,5,PAL,hana_ml.algorithms.pal.trees.GradientBoostingR...,"{""model_attributes"": {""n_estimators"": 101, ""su...",2021-05-20 15:26:21,1


Load model and predict

In [8]:
new_model = storage.load_model('boston', version=1)
new_model.predict(df=test_df)

Creating table with name: AUTOML70c8b478-6008-462b-8d72-83905a5158e3
100%|██████████| 1/1 [00:00<00:00,  2.45it/s]
Preprocessor settings: <hana_automl.preprocess.settings.PreprocessorSettings object at 0x1066e5f70>
Prediction results (first 20 rows): 
     ID    SCORE CONFIDENCE
0    1  34.6182       None
1    2  20.7285       None
2    3  15.3624       None
3    4  25.2115       None
4    5  22.3916       None
5    6  23.9694       None
6    7  20.4695       None
7    8  21.3091       None
8    9  19.0294       None
9   10  41.3506       None
10  11  39.9838       None
11  12  13.5209       None
12  13  10.0027       None
13  14  44.5971       None
14  15  23.6994       None
15  16  12.5791       None
16  17  19.8964       None
17  18   17.987       None
18  19  17.6412       None
19  20  11.2817       None


,ID,SCORE,CONFIDENCE
0,1,34.6182,None
1,2,20.7285,None
2,3,15.3624,None
3,4,25.2115,None
4,5,22.3916,None
...,...,...,...
97,98,25.2976,None
98,99,25.7257,None
99,100,13.6407,None
100,101,40.0866,None


Cleanup storage

In [9]:
storage.clean_up()

For more information, visit AutoML class and Storage class in documentation